In [17]:
import pandas as pd
import numpy as np
import sklearn
import scipy

import anndata as ad
import scanpy as sc

from dask import delayed
from dask.distributed import Client, LocalCluster

import os, binascii, gc

from tqdm.auto import tqdm

Loading expression data, converting to AnnData object

In [18]:
data_dir = '../../datasets'
adata_train_df = pd.read_parquet(os.path.join(data_dir, 'adata_train.parquet'))
adata_obs_meta_df = pd.read_csv(os.path.join(data_dir, 'adata_obs_meta.csv'))
adata_train_df['obs_id'] = adata_train_df['obs_id'].astype('category')
adata_train_df['gene'] = adata_train_df['gene'].astype('category')

obs_ids = adata_train_df['obs_id'].unique()
obs_id_map = dict(zip(obs_ids, range(len(obs_ids))))

genes = adata_train_df['gene'].unique()
gene_map = dict(zip(genes, range(len(genes))))

adata_train_df['obs_index'] = adata_train_df['obs_id'].map(obs_id_map)
adata_train_df['gene_index'] = adata_train_df['gene'].map(gene_map)

normalized_counts_values = adata_train_df['normalized_count'].to_numpy()
counts_values = adata_train_df['count'].to_numpy()

row_indices = adata_train_df['obs_index'].to_numpy()
col_indices = adata_train_df['gene_index'].to_numpy()

counts = scipy.sparse.csr_matrix((counts_values, (row_indices, col_indices)))

obs_df = pd.Series(obs_ids, name='obs_id').to_frame()
var_df = pd.Series(genes, name='gene').to_frame()

obs_df = obs_df.set_index('obs_id')
var_df = var_df.set_index('gene')

obs_df.index = obs_df.index.astype('str')
var_df.index = var_df.index.astype('str')

counts_adata = ad.AnnData(
    X=counts,
    obs=obs_df,
    var=var_df,
    dtype=np.uint32,
)

index_ordering_before_join = counts_adata.obs.index
counts_adata.obs = counts_adata.obs.join(adata_obs_meta_df.set_index('obs_id'))
index_ordering_after_join = counts_adata.obs.index
assert (index_ordering_before_join == index_ordering_after_join).all()

counts_adata

/Applications/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:402: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


AnnData object with n_obs × n_vars = 240090 × 21255
    obs: 'library_id', 'plate_name', 'well', 'row', 'col', 'cell_id', 'donor_id', 'cell_type', 'sm_lincs_id', 'sm_name', 'SMILES', 'dose_uM', 'timepoint_hr', 'control'

In [19]:
counts_adata.write_h5ad('../../datasets/singlecell_adata.h5ad');

Pseudobulking counts by cell type

In [20]:
from scipy import sparse

def sum_by(adata: ad.AnnData, col: str) -> ad.AnnData:
    """
    Adapted from this forum post: 
    https://discourse.scverse.org/t/group-sum-rows-based-on-jobs-feature/371/4
    """
    
    assert pd.api.types.is_categorical_dtype(adata.obs[col])

    # sum `.X` entries for each unique value in `col`
    cat = adata.obs[col].values
    indicator = sparse.coo_matrix(
        (
            np.broadcast_to(True, adata.n_obs),
            (cat.codes, np.arange(adata.n_obs))
        ),
        shape=(len(cat.categories), adata.n_obs),
    )
    sum_adata = ad.AnnData(
        indicator @ adata.X,
        var=adata.var,
        obs=pd.DataFrame(index=cat.categories),
        dtype=adata.X.dtype,
    )
    
    # copy over `.obs` values that have a one-to-one-mapping with `.obs[col]`
    obs_cols = adata.obs.columns
    obs_cols = list(set(adata.obs.columns) - set([col]))
    
    one_to_one_mapped_obs_cols = []
    nunique_in_col = adata.obs[col].nunique()
    for other_col in obs_cols:
        if len(adata.obs[[col, other_col]].drop_duplicates()) == nunique_in_col:
            one_to_one_mapped_obs_cols.append(other_col)

    joining_df = adata.obs[[col] + one_to_one_mapped_obs_cols].drop_duplicates().set_index(col)
    assert (sum_adata.obs.index == sum_adata.obs.join(joining_df).index).all()
    sum_adata.obs = sum_adata.obs.join(joining_df)
    sum_adata.obs.index.name = col
    sum_adata.obs = sum_adata.obs.reset_index()
    sum_adata.obs.index = sum_adata.obs.index.astype('str')

    return sum_adata

In [21]:
counts_adata.obs['plate_well_cell_type'] = counts_adata.obs['plate_name'].astype('str') \
    + '_' + counts_adata.obs['well'].astype('str') \
    + '_' + counts_adata.obs['cell_type'].astype('str')
counts_adata.obs['plate_well_cell_type'] = counts_adata.obs['plate_well_cell_type'].astype('category')

bulk_adata = sum_by(counts_adata, 'plate_well_cell_type')
bulk_adata.obs = bulk_adata.obs.drop(columns=['plate_well_cell_type'])
bulk_adata.X = np.array(bulk_adata.X.todense())
bulk_adata.X = bulk_adata.X.astype('float64')
bulk_adata = bulk_adata.copy()

/Applications/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:402: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


In [22]:
bulk_adata.write_h5ad('../../datasets/raw_bulk_adata.h5ad');

In [23]:
bulk_adata = sc.read_h5ad('../../datasets/raw_bulk_adata.h5ad');
bulk_adata

AnnData object with n_obs × n_vars = 2558 × 21255
    obs: 'row', 'cell_id', 'SMILES', 'cell_type', 'plate_name', 'sm_name', 'donor_id', 'sm_lincs_id', 'timepoint_hr', 'col', 'library_id', 'dose_uM', 'well', 'control'

Loading pseudobulked counts from correctly filtered AnnData: the correctly filtered data contains 18211 genes which are used to generate the de_train.parquet.

In [24]:
fixed_bulk_adata = sc.read_h5ad('../../datasets/train_or_control_bulk_by_cell_type_adata.h5ad')
fixed_bulk_adata.X = fixed_bulk_adata.layers['counts']

In [25]:
new_plate_names = bulk_adata.obs['plate_name'].sort_values().unique()
original_plate_names = fixed_bulk_adata.obs['plate_name'].sort_values().unique()
plate_name_map = dict(zip(original_plate_names, new_plate_names))

fixed_bulk_adata.obs['plate_name'] = fixed_bulk_adata.obs['plate_name'].map(plate_name_map)

In [26]:
new_donor_ids = bulk_adata.obs['donor_id'].sort_values().unique()
original_donor_ids = fixed_bulk_adata.obs['raw_cell_id'].sort_values().unique()
donor_id_map = dict(zip(original_donor_ids, new_donor_ids))

fixed_bulk_adata.obs['donor_id'] = fixed_bulk_adata.obs['raw_cell_id'].map(donor_id_map)

In [27]:
lincs_id_mapping_df = pd.read_parquet('../../datasets/lincs_id_compound_mapping.parquet')

In [28]:
compound_id_to_sm_lincs_id = lincs_id_mapping_df.set_index('compound_id')['sm_lincs_id'].to_dict()
compound_id_to_sm_name = lincs_id_mapping_df.set_index('compound_id')['sm_name'].to_dict()
compound_id_to_smiles = lincs_id_mapping_df.set_index('compound_id')['smiles'].to_dict()

fixed_bulk_adata.obs['sm_lincs_id'] = \
    fixed_bulk_adata.obs['compound_id'].map(compound_id_to_sm_lincs_id)
fixed_bulk_adata.obs['sm_name'] = \
    fixed_bulk_adata.obs['compound_id'].map(compound_id_to_sm_name)
fixed_bulk_adata.obs['SMILES'] = \
    fixed_bulk_adata.obs['compound_id'].map(compound_id_to_smiles)

In [29]:
sorted_index = fixed_bulk_adata.obs.sort_values(['plate_name', 'sm_name', 'cell_type']).index
fixed_bulk_adata = fixed_bulk_adata[sorted_index].copy()
sorted_index = bulk_adata.obs.sort_values(['plate_name', 'sm_name', 'cell_type']).index
bulk_adata = bulk_adata[sorted_index].copy()

In [30]:
bulk_adata

AnnData object with n_obs × n_vars = 2558 × 21255
    obs: 'row', 'cell_id', 'SMILES', 'cell_type', 'plate_name', 'sm_name', 'donor_id', 'sm_lincs_id', 'timepoint_hr', 'col', 'library_id', 'dose_uM', 'well', 'control'

In [31]:
fixed_bulk_adata

AnnData object with n_obs × n_vars = 2558 × 18211
    obs: 'bio_sample_id', 'cell_type', 'raw_cell_id', 'bio_exp_id', 'raw_plate_name', 'compound_id', 'cell_id', 'plate_name', 'dose_uM', 'pert', 'clt_batch_id', 'hashtag_id', 'library_id', 'compound_batch_id', 'cy_id', 'container_format', 'row', 'cy_batch_id', 'compound_name', 'timepoint_hr', 'clt_id', 'col', 'well', 'plate_id', 'readable_pert', 'split', 'donor_id', 'sm_lincs_id', 'sm_name', 'SMILES'
    uns: 'log1p'
    layers: 'counts'

In [32]:
np.allclose(fixed_bulk_adata.X, bulk_adata[:, fixed_bulk_adata.var.index].X)

False

Metadata mismatch

In [33]:
clean_bulk_adata = bulk_adata[:, fixed_bulk_adata.var.index]

In [34]:
clean_bulk_adata

View of AnnData object with n_obs × n_vars = 2558 × 18211
    obs: 'row', 'cell_id', 'SMILES', 'cell_type', 'plate_name', 'sm_name', 'donor_id', 'sm_lincs_id', 'timepoint_hr', 'col', 'library_id', 'dose_uM', 'well', 'control'

In [35]:
pd.merge(
    clean_bulk_adata.obs[['cell_type', 'plate_name', 'donor_id', 'sm_lincs_id']].drop_duplicates().reset_index(),
    fixed_bulk_adata.obs[['cell_type', 'plate_name', 'donor_id', 'sm_lincs_id']].drop_duplicates().reset_index(),
    on=['cell_type', 'plate_name', 'donor_id', 'sm_lincs_id'],
    how='inner'
).shape

(53, 6)

In [36]:
fixed_bulk_adata.obs[['cell_type', 'plate_name', 'donor_id', 'sm_lincs_id']].drop_duplicates().reset_index().shape

(1861, 5)

Debug mismatch

In [37]:
(
    clean_bulk_adata.obs.query('cell_type=="NK cells" and col>3')[['cell_type', 'plate_name', 'donor_id', 'sm_lincs_id']]
    .astype(str)
    .pivot(index='sm_lincs_id', columns='donor_id', values='plate_name')
).drop_duplicates()

donor_id,donor_0,donor_1,donor_2
sm_lincs_id,,,
LSM-1005,plate_2,plate_3,plate_1
LSM-1023,plate_0,plate_5,plate_4
LSM-1025,plate_2,plate_3,NaN
LSM-45410,NaN,NaN,plate_4
LSM-45496,plate_2,NaN,plate_1
LSM-4944,plate_0,plate_5,NaN


In [38]:
(
    fixed_bulk_adata.obs.query('cell_type=="NK cells" and col>3')[['cell_type', 'plate_name', 'donor_id', 'sm_lincs_id']]
    .astype(str)
    .pivot(index='sm_lincs_id', columns='donor_id', values='plate_name')
).drop_duplicates()

donor_id,donor_0,donor_1,donor_2
sm_lincs_id,,,
LSM-1005,plate_0,plate_2,plate_4
LSM-1023,plate_1,plate_3,plate_5
LSM-1025,plate_0,NaN,plate_4
LSM-45410,NaN,plate_3,NaN
LSM-45496,plate_0,plate_2,NaN
LSM-4944,plate_1,NaN,plate_5


Fix mismatch

In [40]:
donor_rename_dict = {
    'donor_0':'donor_0',
    'donor_1':'donor_2',
    'donor_2':'donor_1',
}

plate_rename_dict = {
    'plate_0' : 'plate_2',
    'plate_1' : 'plate_0',
    'plate_2' : 'plate_1',
    'plate_3' : 'plate_4',
    'plate_4' : 'plate_3',
    'plate_5' : 'plate_5',
}

In [41]:
fixed_bulk_adata.obs['donor_id'] = fixed_bulk_adata.obs['donor_id'].map(donor_rename_dict)
fixed_bulk_adata.obs['plate_name'] = fixed_bulk_adata.obs['plate_name'].map(plate_rename_dict)

In [46]:
bulk_adata = fixed_bulk_adata.copy()

NameError: name 'fixed_bulk_adata' is not defined

In [47]:
bulk_adata.write_h5ad('../../datasets/bulk_adata.h5ad');
del fixed_bulk_adata; gc.collect();

In [48]:
de_pert_cols = [
    'sm_name',
    'sm_lincs_id',
    'SMILES',
    'dose_uM',
    'timepoint_hr',
    'cell_type',
]

control_compound = 'Dimethyl Sulfoxide'

In [49]:
cell_rename_dict = {
    'NK cells' : 'NK',
    'T cells CD4+' : 'T4',
    'T cells CD8+' : 'T8',
    'T regulatory cells' : 'Treg',
    'B cells' : 'B',
    'Myeloid cells' : 'Myeloid',
}

In [66]:
bulk_adata.X_df =pd.DataFrame(bulk_adata.X)

Output bulked counts data in the format of csv

In [67]:
bulk_adata.obs.to_csv("../../datasets/bulk_adata_obs.csv")
bulk_adata.var.to_csv("../../datasets/bulk_adata_var.csv")
bulk_adata.X_df.to_csv("../../datasets/bulk_adata_X.csv")

Prepare R code and input data for limma

In [69]:
compound_name_col = de_pert_cols[0]
cell_types = bulk_adata.obs['cell_type'].unique()

In [70]:
with open('r_code.sh', 'w') as f:

    for cell_type in tqdm(cell_types, position=1, desc='cell_type'):
        rename_cell_type = cell_rename_dict[cell_type]
        cell_type_working_dir = 'cell_type_{}'.format(rename_cell_type)

        os.makedirs(cell_type_working_dir, exist_ok=True)

        cell_type_selection = bulk_adata.obs['cell_type'].eq(cell_type)
        cell_type_bulk_adata = bulk_adata[cell_type_selection].copy()

        rpert_mapping = cell_type_bulk_adata.obs[compound_name_col].drop_duplicates() \
        .reset_index(drop=True).reset_index() \
        .set_index(compound_name_col)['index'].to_dict()

        cell_type_bulk_adata.obs['Rpert'] = cell_type_bulk_adata.obs.apply(
            lambda row: rpert_mapping[row[compound_name_col]], 
            axis='columns',
        ).astype('str')

        compound_name_to_Rpert = cell_type_bulk_adata.obs.set_index(compound_name_col)['Rpert'].to_dict()
        ref_pert = compound_name_to_Rpert[control_compound]

        # save h5ad for each cell type
        cell_type_bulk_adata.write_h5ad(os.path.join(cell_type_working_dir, 'input.h5ad'))

        random_string = binascii.b2a_hex(os.urandom(15)).decode()

        gc.collect();

        
        Rscript = '`which Rscript`'

        # for limma fit
        exec_path_fit = 'limma_fit.r'
        design_fit = '~0+Rpert+donor_id+plate_name+row'

        input_path = os.path.join(cell_type_working_dir, 'input.h5ad')
        fit_output_path = os.path.join(cell_type_working_dir, 'limma.rds')
        fit_plot_output_path = os.path.join(cell_type_working_dir, 'voom.pdf')


        r_code_for_fit_in_bash = f'''
echo '# limma fit for cell type {rename_cell_type}'
{Rscript} \
{exec_path_fit} \
--input_h5ad \
{input_path} \
--design \
'{design_fit}' \
--fit_output_path \
{fit_output_path} \
--plot_output_path \
{fit_plot_output_path} 
'''

        print (r_code_for_fit_in_bash, file=f)

        # for limma contrast
        exec_path_contrast = 'limma_contrast.r'

        for pert in cell_type_bulk_adata.obs['Rpert'].unique():
            if pert == ref_pert:
                continue
            else:
                contrast_output_path = os.path.join(cell_type_working_dir, 'pert_{}_contrast_result.csv'.format(pert))
                contrast = 'Rpert'+pert+'-Rpert'+ref_pert
                r_code_for_contrast_in_bash = f'''
echo '# limma contrast for cell type {rename_cell_type} with Rpert={pert}'
{Rscript} \
{exec_path_contrast} \
--input_fit \
{fit_output_path} \
--contrast \
{contrast} \
--contrast_output_path \
{contrast_output_path}
'''

            print (r_code_for_contrast_in_bash, file=f)

cell_type:   0%|          | 0/6 [00:00<?, ?it/s]

In [74]:
! head r_code.sh


echo '# limma fit for cell type B'
`which Rscript` limma_fit.r --input_h5ad cell_type_B/input.h5ad --design '~0+Rpert+donor_id+plate_name+row' --fit_output_path cell_type_B/limma.rds --plot_output_path cell_type_B/voom.pdf 


echo '# limma contrast for cell type B with Rpert=0'
`which Rscript` limma_contrast.r --input_fit cell_type_B/limma.rds --contrast Rpert0-Rpert5 --contrast_output_path cell_type_B/pert_0_contrast_result.csv


echo '# limma contrast for cell type B with Rpert=1'
